In [8]:
tvch = 1
days_ = 20200318

url = f'http://nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu={tvch}_1&area=01&begin_date={days_}'

params = {
    'sYear': '2020',""
    'sMonth': '03',
    'sDay': '08',
}

In [9]:
resp = requests.post(url, headers=params)
soup = BeautifulSoup(resp.content, 'html.parser')
len(soup)

28

In [10]:
tv_ch = {

    1: '지상파',
    2: '종편',
    3: '케이블',
}

In [11]:
elements = soup.select('.ranking_tb')

In [101]:
from datetime import datetime, timedelta

days_range = []

end = datetime.now()
start = (end+timedelta(days = -14))
# end = datetime.now()
date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days)]
days_range = [date_.strftime("%Y%m%d") for date_ in date_generated] 

In [103]:
dates = soup.select_one('.ranking_date').text.strip()
dates

'2020.03.16'

In [930]:
year = datas[0].text.split('\n')[-2]
year

'2020년'

In [28]:
ranks = soup.select('.tb_txt_center')[0].text.split('\t')[0]
ranks

'1'

In [29]:
broadcasts = soup.select('.tb_txt_center')[1].text.split('\t')[0]
broadcasts

'KBS1'

In [31]:
program = soup.select('table.ranking_tb .tb_txt')[0].text.split('\t')[0]
program

'일일연속극(꽃길만걸어요)'

In [33]:
rate = soup.select('table.ranking_tb .percent')[0].text.strip()
rate

'20.3'

In [617]:
elements = soup.select('table.ranking_tb')
top20 = elements[0].get_text().strip().replace('\n', ' ').replace(
    '\r', ' ').replace('\t', ' ').replace('  ', ' ').split(' ')

----
### restart

In [68]:
tv_channels = {
    
    '1':'지상파',
    '2':'종편',
    '3':'케이블',
}

In [82]:
from datetime import datetime, timedelta

def program_percentage(tv_channel,date_):

    url = f'http://nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu={tv_channel}_1&area=01&begin_date={date_}'

    params = {
        'sYear': '2020',
        'sMonth': '03',
        'sDay': '08',
    }
    resp = requests.post(url, headers=params)
    soup = BeautifulSoup(resp.content, 'html.parser')

    days_range = []
    rank_list = []
    pg_list = []
    bs_list = []
    rate_list = []
    ch_list = []
    datas = []
    
    # 날짜
    end = datetime.now()
    start = (end+timedelta(days = -14))
    date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days)]
    days_range = [date_.strftime("%Y%m%d") for date_ in date_generated] 
    
    # columns
    ranks = soup.select('.tb_txt_center')
    broadcasts = soup.select('.tb_txt_center')
    programs = soup.select('table.ranking_tb .tb_txt')
    rates_10 = soup.select('table.ranking_tb .percent')
    rates_20 = soup.select('table.ranking_tb .percent_g')
    rates_all = rates_10[:10]+rates_20[:10]

    rank_list = [rank.text.split('\t')[0] for rank in ranks[0:40:2]]
    bs_list = [broadcast.text.split('\t')[0]
               for broadcast in broadcasts[1:40:2]]
    pg_list = [program.text.split('\t')[0] for program in programs[0:20]]
    rate_list = [rate.text.strip() for rate in rates_all]
        
    datas = pd.DataFrame({'rank': rank_list,
                       'program': pg_list,
                       'broadcast': bs_list,
                       'rate': rate_list,})

    return datas

In [83]:
from datetime import datetime, timedelta

def program_percentage_another(tv_channel,date_):

    url = f'http://nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu={tv_channel}_1&area=01&begin_date={date_}'

    params = {
        'sYear': '2020',
        'sMonth': '03',
        'sDay': '08',
    }
    resp = requests.post(url, headers=params)
    soup = BeautifulSoup(resp.content, 'html.parser')
    
    days_range = []
    rank_list = []
    pg_list = []
    bs_list = []
    rate_list = []
    ch_list = []
    datas_02 = []
    
     # 날짜
    end = datetime.now()
    start = (end+timedelta(days = -14))
    date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days)]
    days_range = [date_.strftime("%Y%m%d") for date_ in date_generated] 
    

    ranks = soup.select('.tb_txt_center')
    broadcasts = soup.select('.tb_txt_center')
    programs = soup.select('table.ranking_tb .tb_txt')
    rates_10 = soup.select('table.ranking_tb .percent')
    rates_20 = soup.select('table.ranking_tb .percent_g')
    rates_all = rates_10[:10]+rates_20[:10]


    rank_list = [rank.text.split('\t')[0] for rank in ranks[0:20:2]]
    bs_list = [broadcast.text.split('\t')[0]
               for broadcast in broadcasts[1:20:2]]
    pg_list = [program.text.split('\t')[0] for program in programs[0:10]]
    rate_list = [rate.text.strip() for rate in rates_all]

    
    datas_02 = pd.DataFrame({'rank': rank_list,
                          'program': pg_list,
                          'broadcast': bs_list,
                          'rate': rate_list,})

    return datas_02

In [ ]:
dfs_01 = []

for tv_channel in range(5):
    for days_ in days_range:
        if tv_channel == 1:
            df_01 = program_percentage(tv_channel,days_)
            dfs_01.append(df_01)
            df_01['category'] = tv_channels['1']
            df_01['date'] = days_
print(dfs_01)

In [ ]:
dfs_02 = []
for tv_channel in range(2,4):
    for days_ in days_range:
        if tv_channel == 2:
            df = program_percentage_another(tv_channel,days_)
            dfs_02.append(df)
            df['category'] = tv_channels['2']
            df['date'] = days_
        else:
            df_ = program_percentage_another(tv_channel,days_)
            dfs_02.append(df_)
            df_['category'] = tv_channels['3']
            df_['date'] = days_
print(dfs_02)

In [86]:
nielsen_df_01 = pd.concat(dfs_01)
nielsen_df_01.tail(2)

,rank,program,broadcast,rate,category,date
18,19,동상이몽2너는내운명,SBS,6.5,지상파,20200316
19,20,생활의달인,SBS,6.3,지상파,20200316


In [87]:
nielsen_df_02 = pd.concat(dfs_02)
nielsen_df_02.tail()

,rank,program,broadcast,rate,category,date
5,6,YTN뉴스특보,YTN,2.077,케이블,20200316
6,7,YTN뉴스특보,YTN,2.049,케이블,20200316
7,8,뉴스특보코로나19COVID19,연합뉴스TV,2.021,케이블,20200316
8,9,뉴스특보코로나19COVID19,연합뉴스TV,1.974,케이블,20200316
9,10,YTN뉴스특보,YTN,1.944,케이블,20200316


In [75]:
nielsen_df_02.to_csv('./0317_종편_케이블_시청률.csv', index=False, encoding='utf-8-sig')

In [76]:
nielsen_df_01.to_csv('./0317_지상파_시청률.csv', index=False, encoding='utf-8-sig')